In [1]:
%matplotlib inline


[파이토치(PyTorch) 기본 익히기](intro.html) ||
[빠른 시작](quickstart_tutorial.html) ||
[텐서(Tensor)](tensorqs_tutorial.html) ||
[Dataset과 Dataloader](data_tutorial.html) ||
[변형(Transform)](transforms_tutorial.html) ||
**신경망 모델 구성하기** ||
[Autograd](autogradqs_tutorial.html) ||
[최적화(Optimization)](optimization_tutorial.html) ||
[모델 저장하고 불러오기](saveloadrun_tutorial.html)

# 신경망 모델 구성하기

신경망은 데이터에 대한 연산을 수행하는 계층(layer)/모듈(module)로 구성되어 있습니다.
[torch.nn](https://pytorch.org/docs/stable/nn.html) 네임스페이스는 신경망을 구성하는데 필요한 모든 구성 요소를 제공합니다.
PyTorch의 모든 모듈은 [nn.Module](https://pytorch.org/docs/stable/generated/torch.nn.Module.html) 의 하위 클래스(subclass)
입니다. 신경망은 다른 모듈(계층; layer)로 구성된 모듈입니다. 이러한 중첩된 구조는 복잡한 아키텍처를 쉽게 구축하고 관리할 수 있습니다.

이어지는 장에서는 FashionMNIST 데이터셋의 이미지들을 분류하는 신경망을 구성해보겠습니다.


In [2]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

## 학습을 위한 장치 얻기

가능한 경우 GPU와 같은 하드웨어 가속기에서 모델을 학습하려고 합니다.
[torch.cuda](https://pytorch.org/docs/stable/notes/cuda.html) 를 사용할 수 있는지
확인하고 그렇지 않으면 CPU를 계속 사용합니다.



In [3]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cuda device


## 클래스 정의하기

신경망 모델을 ``nn.Module`` 의 하위클래스로 정의하고, ``__init__`` 에서 신경망 계층들을 초기화합니다.
``nn.Module`` 을 상속받은 모든 클래스는 ``forward`` 메소드에 입력 데이터에 대한 연산들을 구현합니다.



In [4]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten() # 28 x 28 사이즈의 2D 영상을 1D-array로 펴줌
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

``NeuralNetwork`` 의 인스턴스(instance)를 생성하고 이를 ``device`` 로 이동한 뒤,
구조(structure)를 출력합니다.



In [5]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


모델을 사용하기 위해 입력 데이터를 전달합니다. 이는 일부
[백그라운드 연산들](https://github.com/pytorch/pytorch/blob/270111b7b611d174967ed204776985cefca9c144/torch/nn/modules/module.py#L866) 과 함께
모델의 ``forward`` 를 실행합니다. ``model.forward()`` 를 직접 호출하지 마세요!

모델에 입력을 전달하여 호출하면 2차원 텐서를 반환합니다. 2차원 텐서의 dim=0은 각 분류(class)에 대한 원시(raw) 예측값 10개가,
dim=1에는 각 출력의 개별 값들이 해당합니다.
원시 예측값을 ``nn.Softmax`` 모듈의 인스턴스에 통과시켜 예측 확률을 얻습니다.



In [7]:
X = torch.rand(1, 28, 28, device=device)
logits = model(X)
print(logits)
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")

tensor([[-0.0156, -0.0795,  0.0161, -0.0114, -0.0400, -0.0643, -0.0034,  0.0533,
         -0.0340, -0.0471]], device='cuda:0', grad_fn=<AddmmBackward0>)
Predicted class: tensor([7], device='cuda:0')


------------------------------------------------------------------------------------------




## 모델 계층(Layer)

FashionMNIST 모델의 계층들을 살펴보겠습니다. 이를 설명하기 위해, 28x28 크기의 이미지 3개로 구성된
미니배치를 가져와, 신경망을 통과할 때 어떤 일이 발생하는지 알아보겠습니다.



In [8]:
input_image = torch.rand(3,28,28)
print(input_image.size())

torch.Size([3, 28, 28])


### nn.Flatten
[nn.Flatten](https://pytorch.org/docs/stable/generated/torch.nn.Flatten.html) 계층을 초기화하여
각 28x28의 2D 이미지를 784 픽셀 값을 갖는 연속된 배열로 변환합니다. (dim=0의 미니배치 차원은 유지됩니다.)



In [9]:
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

torch.Size([3, 784])


### nn.Linear
[선형 계층](https://pytorch.org/docs/stable/generated/torch.nn.Linear.html) 은 저장된 가중치(weight)와
편향(bias)을 사용하여 입력에 선형 변환(linear transformation)을 적용하는 모듈입니다.




In [10]:
layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


### nn.ReLU
비선형 활성화(activation)는 모델의 입력과 출력 사이에 복잡한 관계(mapping)를 만듭니다.
비선형 활성화는 선형 변환 후에 적용되어 *비선형성(nonlinearity)* 을 도입하고, 신경망이 다양한 현상을 학습할 수 있도록 돕습니다.

이 모델에서는 [nn.ReLU](https://pytorch.org/docs/stable/generated/torch.nn.ReLU.html) 를 선형 계층들 사이에 사용하지만,
모델을 만들 때는 비선형성을 가진 다른 활성화를 도입할 수도 있습니다.



In [11]:
print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")

Before ReLU: tensor([[ 0.3479,  0.4362,  0.2055,  0.2411, -0.4391,  0.1949,  0.1125,  0.2736,
          0.2424, -0.2306, -0.2387, -0.4553, -0.7706, -0.5570,  0.2937,  0.2299,
         -0.1386,  0.1589, -0.0865, -0.5016],
        [ 0.4820,  0.9982,  0.1814,  0.3137, -0.4738, -0.3453,  0.0680,  0.0723,
          0.4505,  0.0122, -0.2619, -0.8509, -0.5474,  0.0341,  0.2302,  0.3691,
         -0.0155,  0.1964, -0.0416, -0.2606],
        [ 0.3692,  0.7591,  0.0870,  0.0038, -0.2385,  0.0690, -0.0022,  0.2187,
          0.4788, -0.2463,  0.1497, -0.4389, -0.1630, -0.1830,  0.1483,  0.2249,
          0.1629,  0.4220,  0.1411, -0.7104]], grad_fn=<AddmmBackward0>)


After ReLU: tensor([[0.3479, 0.4362, 0.2055, 0.2411, 0.0000, 0.1949, 0.1125, 0.2736, 0.2424,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.2937, 0.2299, 0.0000, 0.1589,
         0.0000, 0.0000],
        [0.4820, 0.9982, 0.1814, 0.3137, 0.0000, 0.0000, 0.0680, 0.0723, 0.4505,
         0.0122, 0.0000, 0.0000, 0.0000, 0.0341, 0.23

### nn.Sequential
[nn.Sequential](https://pytorch.org/docs/stable/generated/torch.nn.Sequential.html) 은 순서를 갖는
모듈의 컨테이너입니다. 데이터는 정의된 것과 같은 순서로 모든 모듈들을 통해 전달됩니다. 순차 컨테이너(sequential container)를 사용하여
아래의 ``seq_modules`` 와 같은 신경망을 빠르게 만들 수 있습니다.



In [16]:
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20, 10)
)
input_image = torch.rand(3,28,28)
logits = seq_modules(input_image)
print(logits)

tensor([[ 0.1556, -0.0008, -0.1094,  0.1675, -0.1548, -0.1344, -0.0982,  0.2332,
          0.2026, -0.1551],
        [ 0.2703, -0.0760,  0.0507,  0.1515, -0.2697, -0.0660, -0.0220,  0.2489,
          0.2212, -0.1735],
        [ 0.2423, -0.0489, -0.0299,  0.1320, -0.1343, -0.0502, -0.0749,  0.1739,
          0.2080, -0.1680]], grad_fn=<AddmmBackward0>)


### nn.Softmax
신경망의 마지막 선형 계층은 [nn.Softmax](https://pytorch.org/docs/stable/generated/torch.nn.Softmax.html) 모듈에 전달될
([-\infty, \infty] 범위의 원시 값(raw value)인) `logits` 를 반환합니다. logits는 모델의 각 분류(class)에 대한 예측 확률을 나타내도록
[0, 1] 범위로 비례하여 조정(scale)됩니다. ``dim`` 매개변수는 값의 합이 1이 되는 차원을 나타냅니다.



In [15]:
softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)
print(pred_probab)

tensor([[0.0824, 0.0719, 0.1291, 0.0952, 0.0922, 0.1206, 0.1160, 0.1055, 0.0902,
         0.0970],
        [0.0974, 0.0778, 0.1052, 0.1043, 0.1003, 0.1330, 0.0811, 0.1131, 0.0915,
         0.0964],
        [0.0961, 0.0839, 0.1189, 0.1194, 0.0872, 0.1189, 0.0873, 0.0968, 0.0996,
         0.0919]], grad_fn=<SoftmaxBackward0>)


## 모델 매개변수

신경망 내부의 많은 계층들은 *매개변수화(parameterize)* 됩니다. 즉, 학습 중에 최적화되는 가중치와 편향과 연관지어집니다.
``nn.Module`` 을 상속하면 모델 객체 내부의 모든 필드들이 자동으로 추적(track)되며, 모델의 ``parameters()`` 및
``named_parameters()`` 메소드로 모든 매개변수에 접근할 수 있게 됩니다.

이 예제에서는 각 매개변수들을 순회하며(iterate), 매개변수의 크기와 값을 출력합니다.




In [23]:
print(f"Model structure: {model}\n\n")

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")

Model structure: NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values : tensor([[-0.0167, -0.0056,  0.0033,  ...,  0.0180, -0.0230, -0.0065],
        [-0.0009, -0.0211,  0.0356,  ...,  0.0258, -0.0137,  0.0322]],
       device='cuda:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values : tensor([-0.0192,  0.0339], device='cuda:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values : tensor([[-0.0183,  0.0061,  0.0282,  ...,  0.0147, -0.0395, -0.0106],
        [ 0.0171, -0.0065,  0.0233,  ..., -0.0303, -0.0385, -0.0052]],
       device='cuda:0', grad_fn=<Sl

------------------------------------------------------------------------------------------




## 더 읽어보기
- [torch.nn API](https://pytorch.org/docs/stable/nn.html)

